In [119]:
## Once yfinance is installed, you can use the following code to download historical data for your 7 assets.

# Load libraries
from IPython.display import display
import pandas as pd
import yfinance as yf
import datetime as dt
import numpy as np
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
import statistics

# Create a list of assets (update this with your chosen assets):
stock_tickers = ['C', 'DB', 'GE', 'GM', 'MS', 'NVDA', 'RYCEY']
fund_tickers = ['DIA', 'IWM', 'SPY']
full_tickers = ['C', 'DB', 'GE', 'GM', 'MS', 'NVDA', 'RYCEY', 'DIA',  'SPY', 'IWM']

# Assign a start date and end date for the data download, 1 year of data
start_date = dt.datetime.today() - pd.Timedelta(3650, "d")
end_date = dt.datetime.today()

# Download the data using the above variables
stock_port = yf.download(start=start_date, end=end_date, tickers = stock_tickers)['Adj Close'].sort_values(by='Date',ascending=False)
fund_port = yf.download(start=start_date, end=end_date, tickers = fund_tickers)['Adj Close'].sort_values(by='Date',ascending=False)
full_port = yf.download(start=start_date, end=end_date, tickers = full_tickers)['Adj Close'].sort_values(by='Date',ascending=False)


[*********************100%%**********************]  7 of 7 completed
[*********************100%%**********************]  3 of 3 completed
[*********************100%%**********************]  10 of 10 completed


In [137]:
###PART ONE###


# Portfolio Weight (equally weighted)
portfolio_weight=1/len(stock_tickers)

# Annualized Volatility (using trailing 3-months)
three_month_adj_close = stock_port.head(63)
standard_deviation=three_month_adj_close.std()/three_month_adj_close.mean()*np.sqrt(253)

annualized_volatility=standard_deviation.tolist()


# Beta against SPY (using trailing 12-months)
covariance=(full_port.head(253).pct_change().cov().drop(['SPY', 'DIA', 'IWM'])[['DIA', 'IWM', 'SPY']])
variance=(full_port.head(253).pct_change().var()[['DIA', 'IWM', 'SPY']])

variance_dia=(variance['DIA'])
variance_iwm=(variance['IWM'])
variance_spy=(variance['SPY'])

covariance=covariance.div([variance_dia, variance_iwm, variance_spy])

beta_dia=covariance['DIA']
beta_iwm=covariance['IWM']
beta_spy=covariance['SPY']

# Average and Max Weekly Drawdown
stock_port = stock_port.sort_values(by='Date',ascending=True)
week_dd = list() # weekly drawdown
week_avg = list() # average of drawdowns
week_max = list() # max drawdowns
start_date = dt.date.today()
for y in range(len(stock_tickers)): # Loop over tickers
    for z in range(1,52): # Loop over 52 weeks
        week_dd.append((stock_port[stock_tickers[y]].loc[start_date-dt.timedelta(days=6):start_date].min()-stock_port[stock_tickers[y]].loc[start_date-dt.timedelta(days=6):start_date].max())/stock_port[stock_tickers[y]].loc[start_date-dt.timedelta(days=6):start_date].max()) # Appending weekly drawdown to a list 
        start_date = start_date - dt.timedelta(days=6) # Going to the next 7 day interval
    week_avg.append(statistics.mean(week_dd)) # Taking averages and adding to a list. Individual values correspond to average weekly drawdown of each stock.
    week_max.append(min(week_dd))
    week_dd = list() # Initializing
    start_date = dt.date.today() # Initializing to repeat the same procedure with another stock
stock_port = stock_port.sort_values(by='Date',ascending=False)

# Total Return (using trailing 10-years)
total_retun_1 = (((stock_port.head(1)-(stock_port.iloc[-1]))/(stock_port.iloc[-1]))*100)
total_retun_10 = total_retun_1[str(total_retun_1.columns[0])]

# Annualized Total Return (using trailing 10-years)
annualized_retun_10 =(pow((total_retun_10/100+1),(1/10))-1)*100

# Creating Table
data = {"Portfolio Weight": portfolio_weight, "Annualized Voltility": annualized_volatility, "Beta (DIA)": beta_dia, "Beta (IWM)": beta_iwm, "Beta (SPY)": beta_spy, "Weekly Drawdown (52 Week avg)": week_avg, "Weekly Drawdown (52 Week max)": week_max, "Total Return (%)": total_retun_10, "Total Return (Annualized in %)": annualized_retun_10}
table = pd.DataFrame(data)
display(table)

,Portfolio Weight,Annualized Voltility,Beta (DIA),Beta (IWM),Beta (SPY),Weekly Drawdown (52 Week avg),Weekly Drawdown (52 Week max),Total Return (%),Total Return (Annualized in %)
Ticker,,,,,,,,,
C,0.142857,0.870706,1.447287,0.809694,1.098034,-0.031167,-0.090605,42.304061,3.590929
DB,0.142857,0.498814,1.535863,0.846214,1.271922,-0.035843,-0.090383,-66.036080,-10.236086
GE,0.142857,1.313382,1.046653,0.451694,0.907608,-0.026817,-0.063985,19.602682,1.806168
GM,0.142857,1.003413,1.651753,0.933279,1.277414,-0.040819,-0.121792,43.857600,3.703468
MS,0.142857,0.695415,1.632207,0.861690,1.263985,-0.032131,-0.095654,250.230181,13.353610
NVDA,0.142857,3.038811,1.221290,0.519933,1.991510,-0.055899,-0.195945,18794.466879,68.901484
RYCEY,0.142857,1.137900,1.425206,0.700252,1.256742,-0.047034,-0.212245,-68.990806,-11.049366


In [243]:
### PART TWO ###


# Correltion against ETF

portfolio_sum = (stock_port.pct_change()).sum(axis = 'columns')/len(stock_tickers)
etf_pct = (fund_port.pct_change())
correlation = [portfolio_sum.corr(etf_pct['DIA']), portfolio_sum.corr(etf_pct['IWM']), portfolio_sum.corr(etf_pct['SPY'])]                          

# Covariance against ETF (x10^3 to gain more insight)
covariance = [portfolio_sum.cov(etf_pct["DIA"])*1000, portfolio_sum.cov(etf_pct["IWM"])*1000,portfolio_sum.cov(etf_pct["SPY"])*1000]

# Tracking Errors
a = {"Portfolio": portfolio_sum, 'DIA': etf_pct['DIA'],'IWM': etf_pct['IWM'],'SPY': etf_pct['SPY']}
a = pd.DataFrame(a).iloc[1:, :]

dia_tracking_errr = (a['Portfolio']-a['DIA']).std()*100
iwm_tracking_errr = (a['Portfolio']-a['IWM']).std()*100
spy_tracking_errr = (a['Portfolio']-a['SPY']).std()*100

tracking_error = [dia_tracking_errr, iwm_tracking_errr, spy_tracking_errr]

# Sharpe Ratio

R_f = ((yf.download("^IRX")["Adj Close"].iloc[-1:])+1)**(1/252)-1# gettin risk-free
R_p=portfolio_sum.mean()
Sr=pd.DataFrame([(R_f-R_p)/((portfolio_sum-etf_pct['DIA']).std()),(R_f-R_p)/((portfolio_sum-etf_pct['IWM']).std()),(R_f-R_p)/((portfolio_sum-etf_pct['SPY']).std())]).iloc[:,0]

# Anualized Volatility
P_v = portfolio_sum.head(252).std()* np.sqrt(252)
DIA_v = etf_pct['DIA'].head(252).std()* np.sqrt(252)
IWM_v = etf_pct['IWM'].head(252).std()* np.sqrt(252)
SPY_v = etf_pct['SPY'].head(252).std()* np.sqrt(252)
a_v = [P_v-DIA_v, P_v-IWM_v, P_v-SPY_v]

# Creating Table
table = {"Ticker": fund_tickers, "ETF Correlation": correlation, "ETF Covariance (x10^3)": covariance, "Tracking Error (%)": tracking_error, "Sharpe Ratio": Sr}
table = pd.DataFrame(table)
display(table)



[*********************100%%**********************]  1 of 1 completed


,Ticker,ETF Correlation,ETF Covariance (x10^3),Tracking Error (%),Sharpe Ratio
Adj Close,DIA,0.830983,0.161197,1.037135,0.703481
Adj Close,IWM,0.824848,0.204619,0.992304,0.735263
Adj Close,SPY,0.823560,0.160708,1.049075,0.695474


In [259]:
# Correlation Matrix
ps = pd.DataFrame({"Portfolio": portfolio_sum})
table = pd.merge(full_port.pct_change(),ps, on="Date").iloc[2:]
CM = table.corr()
display(CM)


,C,DB,DIA,GE,GM,IWM,MS,NVDA,RYCEY,SPY,Portfolio
C,1.000000,0.634922,0.774413,0.579203,0.619049,0.744916,0.824823,0.374342,0.359949,0.730062,0.832122
DB,0.634922,1.000000,0.570972,0.443718,0.470348,0.567505,0.634586,0.318284,0.332495,0.551000,0.744738
DIA,0.774413,0.570972,1.000000,0.609131,0.642977,0.850479,0.794051,0.550613,0.372062,0.960089,0.831026
GE,0.579203,0.443718,0.609131,1.000000,0.496308,0.582262,0.551875,0.315552,0.319776,0.577541,0.705121
GM,0.619049,0.470348,0.642977,0.496308,1.000000,0.666964,0.595284,0.374502,0.326869,0.624988,0.742584
IWM,0.744916,0.567505,0.850479,0.582262,0.666964,1.000000,0.753360,0.560189,0.386493,0.875553,0.824848
MS,0.824823,0.634586,0.794051,0.551875,0.595284,0.753360,1.000000,0.441839,0.325649,0.764603,0.827870
NVDA,0.374342,0.318284,0.550613,0.315552,0.374502,0.560189,0.441839,1.000000,0.180399,0.650423,0.609089
RYCEY,0.359949,0.332495,0.372062,0.319776,0.326869,0.386493,0.325649,0.180399,1.000000,0.350128,0.619253
SPY,0.730062,0.551000,0.960089,0.577541,0.624988,0.875553,0.764603,0.650423,0.350128,1.000000,0.823560
